In [55]:
# import libraries
import pandas as pd
import re
import nltk


#### Date Cleaning:
Selected articles were indicated, but irrelevant articles were not. 

In [33]:
# Create new column names because text file has no header
col_names = ["Type", "Author", "Year", "Title", "Journal Name", "Volume", "Issue", "Pages", "URL", "Keywords", "Abstract", "DOI", "PDF Name"]

# Convert text files into a dataframe
silver_yes = pd.read_csv('Hypopthalmichthys_molitrix_yes.txt', sep = '\t', header = None, dtype = str, names = col_names, quotechar = '"')
silver_all = pd.read_csv('Hypopthalmichthys_molitrix_all.txt', sep = '\t', header = None, dtype = str, names = col_names, quotechar = '"')
bighead_yes = pd.read_csv('Hypopthalmichthys_nobilis_yes.txt', sep = '\t', header = None, dtype = str, names = col_names, quotechar = '"')
bighead_all = pd.read_csv('Hypopthalmichthys_nobilis_all.txt', sep = '\t', header = None, dtype = str, names = col_names, quotechar = '"')

# Check entries of selected articles
print(len(silver_yes))
print(len(silver_all))
print(len(bighead_yes))
print(len(bighead_all))


123
444
110
197


In [34]:
# Combine data with selected articles
carp_yes = pd.concat([silver_yes, bighead_yes], ignore_index = True)

# Combine all article - duplicates removed in next step to get irrelevant articles
carp_concat = pd.concat([silver_yes, silver_all, bighead_yes, bighead_all], ignore_index = True)

print(len(carp_yes))
print(len(carp_concat))


233
874


In [35]:
# Drop duplicates to get unselected articles
carp_no = carp_concat.drop_duplicates(keep = False, ignore_index = True)

print(len(carp_no))


408


In [41]:
# subset and select by columns
columns = ["Author", "Year", "Title", "Journal Name", "Volume", "Issue", "Pages", "Abstract"]

carp_yes = carp_yes[columns]
carp_no = carp_no[columns]


In [48]:
# add category columns
carp_yes['categories'] = 'yes'
carp_no['categories'] = 'no'


In [52]:
# combine df
carp_all = pd.concat([carp_yes, carp_no], ignore_index = True)

print(len(carp_all))

641


In [51]:
# Export to csv files
carp_all.to_csv('hypopthalmichthys_selected_articles.csv', index = False)


#### Data Prep

In [53]:
# Import csv files
carp_file = pd.read_csv('hypopthalmichthys_selected_articles.csv')

print(len(carp_file))


641


In [54]:
# Drop any abstracts with NAs
carp_file.dropna(subset = ['Abstract'], inplace = True)

print(len(carp_file))


641


#### Text Processing